In [151]:
import os
import shutil
import pandas as pd


In [159]:
# Define the path to the data folder
dataset_folder = "/Users/karmasmac/Downloads/SteeringModelKarma/Dataset"

# Define the paths to the image and CSV files
center_images_folder = os.path.join(dataset_folder, "center")
right_images_folder = os.path.join(dataset_folder, "right")
left_images_folder = os.path.join(dataset_folder, "left")
steering_data_path = os.path.join(dataset_folder, "steering.csv")
brake_data_path = os.path.join(dataset_folder, "brake.csv")
throttle_data_path = os.path.join(dataset_folder, "throttle.csv")

final_images_folder_path = os.path.join(dataset_folder, 'final_images')
if not os.path.exists(final_images_folder_path):
        os.mkdir(final_images_folder_path)

# Read the CSV files
steering_data = pd.read_csv(steering_data_path)
brake_data = pd.read_csv(brake_data_path)
throttle_data = pd.read_csv(throttle_data_path)


In [160]:

steering_data['timestamp']  = steering_data['timestamp'].astype(str).str[:-7]
throttle_data['timestamp'] =  throttle_data['timestamp'].astype(str).str[:-7]
brake_data['timestamp'] = brake_data['timestamp'].astype(str).str[:-7]

print(steering_data)
print(throttle_data)
print(brake_data)


          timestamp     angle  torque      speed
0      147942421478  0.003491  0.1250  22.880556
1      147942421480  0.003491  0.1250  22.886110
2      147942421482  0.003491  0.1250  22.891666
3      147942421484  0.003491  0.1875  22.897223
4      147942421486  0.003491  0.0000  22.902779
...             ...       ...     ...        ...
11035  147942443586 -0.340339 -1.4375  19.755556
11036  147942443588 -0.340339 -1.3750  19.747223
11037  147942443590 -0.336849 -1.2500  19.738890
11038  147942443592 -0.333358 -1.1875  19.736111
11039  147942443594 -0.328122 -1.3750  19.727777

[11040 rows x 4 columns]
          timestamp  throttle_input
0      147942421478        0.415427
1      147942421480        0.414939
2      147942421482        0.414450
3      147942421484        0.415427
4      147942421486        0.414328
...             ...             ...
11035  147942443583        0.291600
11036  147942443585        0.294530
11037  147942443587        0.300145
11038  147942443589       

In [161]:
steering_ts_ls = steering_data['timestamp'].tolist()
steering_angle_ls = steering_data['angle'].tolist()
throttle_ts_ls = throttle_data['timestamp'].tolist()
throttle_data_ls = throttle_data['throttle_input'].tolist()
brake_ts_ls = brake_data['timestamp'].tolist()
brake_data_ls = brake_data['brake_input'].tolist()

In [163]:
# Iterate over each image file in the images folder
def make_dataset_from_image_path(path):
    timestamp_list = []
    angle_list = []
    throttle_list = []
    brake_list = []
    for filename in os.listdir(path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_name_as_ts = os.path.splitext(filename)[0][:-7]
            if image_name_as_ts in steering_ts_ls and  image_name_as_ts in throttle_ts_ls and image_name_as_ts in brake_ts_ls:

                # Copy the file to the finalImages folder
                source_file = os.path.join(path, filename)
                destination_file = os.path.join(final_images_folder_path, image_name_as_ts + ".png")
                shutil.copy(source_file, destination_file)

                steering_row = steering_data[steering_data['timestamp'] == image_name_as_ts]
                throttle_row = throttle_data[throttle_data['timestamp'] == image_name_as_ts]
                brake_row =  brake_data[brake_data['timestamp'] == image_name_as_ts]
            
                def find_data(row, cname):
                    if not row.empty:
                        data = row[cname].iloc[0]
                        return data

                angle = find_data(steering_row, 'angle')
                throttle = find_data(throttle_row, 'throttle_input')
                brake = find_data(brake_row, 'brake_input')

                timestamp_list.append(image_name_as_ts)
                angle_list.append(angle)
                throttle_list.append(throttle)
                brake_list.append(brake)

            
    return pd.DataFrame({'timestamp': timestamp_list, 'angle': angle_list, 'throttle': throttle_list, 'brake': brake_list})

center = make_dataset_from_image_path(center_images_folder)
right = make_dataset_from_image_path(right_images_folder)
left = make_dataset_from_image_path(left_images_folder)

final_data = pd.concat([center, left, right], ignore_index=True)

# Write the final DataFrame to a CSV file
final_data.to_csv(os.path.join(dataset_folder,'final_data.csv'), index=False)


        timestamp     angle  throttle     brake
0    147942442726 -0.178024  0.232136  0.147372
1    147942436580 -0.144862  0.249958  0.149142
2    147942441606 -0.097738  0.149950  0.161227
3    147942437990 -0.118682  0.233478  0.148547
4    147942440456 -0.101229  0.150072  0.148211
..            ...       ...       ...       ...
649  147942439661  0.001745  0.332509  0.148775
650  147942424583  0.097738  0.391745  0.148394
651  147942437165 -0.137881  0.162280  0.148638
652  147942441496 -0.089012  0.260716  0.147906
653  147942437980 -0.118682  0.231769  0.148363

[654 rows x 4 columns]
        timestamp     angle  throttle     brake
0    147942434809  0.001745  0.374151  0.148775
1    147942436524 -0.151844  0.326528  0.148333
2    147942435889 -0.120428  0.293065  0.148180
3    147942437850 -0.127409  0.149950  0.148241
4    147942442245  0.029671  0.192813  0.147494
..            ...       ...       ...       ...
660  147942436774 -0.113446  0.196963  0.148379
661  14794243485

In [164]:
center

,timestamp,angle,throttle,brake
0,147942442726,-0.178024,0.232136,0.147372
1,147942436580,-0.144862,0.249958,0.149142
2,147942441606,-0.097738,0.149950,0.161227
3,147942437990,-0.118682,0.233478,0.148547
4,147942440456,-0.101229,0.150072,0.148211
...,...,...,...,...
649,147942439661,0.001745,0.332509,0.148775
650,147942424583,0.097738,0.391745,0.148394
651,147942437165,-0.137881,0.162280,0.148638
652,147942441496,-0.089012,0.260716,0.147906


In [165]:
left

,timestamp,angle,throttle,brake
0,147942434809,0.001745,0.374151,0.148775
1,147942436524,-0.151844,0.326528,0.148333
2,147942435889,-0.120428,0.293065,0.148180
3,147942437850,-0.127409,0.149950,0.148241
4,147942442245,0.029671,0.192813,0.147494
...,...,...,...,...
660,147942436774,-0.113446,0.196963,0.148379
661,147942434859,0.001745,0.292454,0.149065
662,147942437145,-0.136136,0.172900,0.148562
663,147942422067,0.010472,0.398581,0.148623


In [166]:
right

,timestamp,angle,throttle,brake
0,147942440170,-0.111701,0.299168,0.147112
1,147942442171,-0.068068,0.160449,0.147250
2,147942439535,0.006981,0.300877,0.148913
3,147942439635,-0.003491,0.332387,0.148196
4,147942438920,0.006981,0.257298,0.148455
...,...,...,...,...
658,147942438365,-0.017453,0.240925,0.148272
659,147942431129,0.013963,0.275486,0.147829
660,147942441696,-0.095993,0.150072,0.148226
661,147942438445,-0.008727,0.168750,0.148714
